<center><img src="../logo.png" alt="Header" style="width: 800px;"/></center>


@Copyright (C): 2010-2020, Shenzhen Yahboom Tech  
@Author: Liusen  
@Date: 2020-02-24 11:10:02  
@LastEditors: Liusen  
@LastEditTime: 2020-02-24 11:10:02    

In [ ]:
#bgr8转jpeg格式
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

# 定义手势识别函数部分

In [ ]:
import cv2 
import time
import demjson
import pygame 
from aip import AipBodyAnalysis
from aip import AipSpeech
from PIL import Image, ImageDraw, ImageFont
import numpy
import ipywidgets.widgets as widgets

# 具体手势请看官方提供 https://ai.baidu.com/ai-doc/BODY/4k3cpywrv
hand={'One':'数字1','Five':'数字5','Fist':'拳头','Ok':'OK',
      'Prayer':'祈祷','Congratulation':'作揖','Honour':'作别',
      'Heart_single':'比心心','Thumb_up':'点赞','Thumb_down':'Diss',
      'ILY':'我爱你','Palm_up':'掌心向上','Heart_1':'双手比心1',
      'Heart_2':'双手比心2','Heart_3':'双手比心3','Two':'数字2',
      'Three':'数字3','Four':'数字4','Six':'数字6','Seven':'数字7',
      'Eight':'数字8','Nine':'数字9','Rock':'Rock','Insult':'竖中指','Face':'脸'}

# 下面的key要换成自己的 
""" 人体分析 APPID AK SK """
APP_ID = 'xxxx'
API_KEY = 'xxxxx'
SECRET_KEY = 'xxxxx'

#camera = PiCamera()
client = AipBodyAnalysis(APP_ID, API_KEY, SECRET_KEY)

g_camera = cv2.VideoCapture(0)
g_camera.set(3, 640)
g_camera.set(4, 480)
g_camera.set(5, 30)  #设置帧率
g_camera.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
g_camera.set(cv2.CAP_PROP_BRIGHTNESS, 60) #设置亮度 -64 - 64  0.0
g_camera.set(cv2.CAP_PROP_CONTRAST, 50) #设置对比度 -64 - 64  2.0
g_camera.set(cv2.CAP_PROP_EXPOSURE, 156) #设置曝光值 1.0 - 5000  156.0

ret, frame = g_camera.read()

# 定义摄像头显示组件

In [ ]:
image_widget = widgets.Image(format='jpeg', width=640, height=480)  #设置摄像头显示组件
display(image_widget)   
image_widget.value = bgr8_to_jpeg(frame)

# 定义转换显示中文函数

In [ ]:
def cv2ImgAddText(img, text, left, top, textColor=(0, 255, 0), textSize=20):
    if (isinstance(img, numpy.ndarray)):  # 判断是否OpenCV图片类型
        img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # 创建一个可以在给定图像上绘图的对象
    draw = ImageDraw.Draw(img)
    # 字体的格式
    fontStyle = ImageFont.truetype(
        "simhei.ttf", textSize, encoding="utf-8")
    # 绘制文本
    draw.text((left, top), text, textColor, font=fontStyle)
    # 转换回OpenCV格式
    return cv2.cvtColor(numpy.asarray(img), cv2.COLOR_RGB2BGR)

# 主进程

In [ ]:
while True:
    """拍照 """
    ret, frame = g_camera.read()
    #retval, frame =   g_camera.read()
    """ 调用手势识别 """
    raw = str(client.gesture(image_widget.value))
    text = demjson.decode(raw)
    try:
        res = text['result'][0]['classname']
    except:
        #print('识别结果：什么也没识别到哦~' )
        # 1 dst 2 文字内容 3 坐标 4 5 字体大小 6 color 7 粗细 8 line type
#       cv2.putText(frame, '未识别', (250,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,200), 2, cv2.LINE_AA) #只能显示英文
        img = cv2ImgAddText(frame, "未识别", 250, 30, (0, 0 , 255), 30)
    else:
        print('识别结果：' + hand[res])
        #cv2.putText(frame, hand[res], (250,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
        img = cv2ImgAddText(frame, hand[res], 250, 30, (0, 255 , 0), 30)

    image_widget.value = bgr8_to_jpeg(img)
          

In [ ]:
g_camera.release()#最后需要释放掉摄像头的占用